In [7]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import datetime
from datetime import datetime
import json
from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
database = 'sqlite:///database1.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy(app)

class User(db.Model):
    __tablename__ = 'users'

    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):
            dob = datetime.today()
        self._dob = dob

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = name

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, uid):
        self._uid = uid

    def is_uid(self, uid):
        return self._uid == uid

    @property
    def password(self):
        return self._password[0:10] + "..."

    def set_password(self, password):
        self._password = generate_password_hash(password, method='sha256')

    def is_password(self, password):
        result = check_password_hash(self._password, password)
        return result

    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string

    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):
            dob = datetime.today()
        self._dob = dob

    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))

    def __str__(self):
        return json.dumps(self.read())

    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None

    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    def update(self, name="", uid="", password=""):
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

def initUsers():
    with app.app_context():
        db.create_all()
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))

        users = [u1, u2, u3, u4, u5, u6]

        for user in users:
            try:
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:
                print(f"Records exist uid {user.uid}, or error.")

initUsers()

def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user
  

def check_credentials(uid, password):
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False



def delete_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
        if user:
            user.delete()
            print(f"User with UID {uid} has been deleted.")
        else:
            print(f"User with UID {uid} not found.")

def create():
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass
    
    name = input("Enter your name:")
    password = input("Enter your password")
    
    user = User(name=name, uid=uid, password=password)
    
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:
            print(f"Unknown error uid {uid}")

    delete_choice = input("Do you want to delete a user? (yes/no): ").lower()

    if delete_choice == "yes":
        uid_to_delete = input("Enter the UID of the user you want to delete: ")
        delete_by_uid(uid_to_delete)
    elif delete_choice == "no":
        print("No user will be deleted.")
    else:
        print("Invalid input. No user will be deleted.")
        
create()



Records exist uid toby, or error.
Records exist uid niko, or error.
Records exist uid lex, or error.
Records exist uid whit, or error.
Created new uid indi
Records exist uid raven, or error.
User with uid indi has been deleted.
Found
 {'id': 9, 'name': 'tej', 'uid': 'ok', 'dob': '03-19-2023', 'age': 0}
